In [1]:
import xarray as xr
import distributed


In [2]:
cluster = distributed.LocalCluster(n_workers=4, threads_per_worker=1, local_directory='/tmp')
client = distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:34321 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.62 GB


# Load IFS data

In [3]:
ds_ifs_pl = xr.open_mfdataset('../data/raw/ifs/pl_*.nc', parallel=True, chunks={'time': 2})
ds_ifs_sfc = xr.open_mfdataset('../data/raw/ifs/sfc_*.nc', parallel=True, chunks={'time': 2})

In [4]:
ds_ifs_t = ds_ifs_pl['t'].to_dataset('level')
ds_ifs_t = ds_ifs_t.rename({500: 't_500', 850: 't_850'})

In [5]:
ds_ifs_gh = ds_ifs_pl['gh'].to_dataset('level')
ds_ifs_gh = ds_ifs_gh.rename({500: 'gh_500', 850: 'gh_850'})

In [6]:
ds_ifs_merged = xr.merge([ds_ifs_sfc, ds_ifs_t, ds_ifs_gh])

In [7]:
ds_ifs_merged = ds_ifs_merged.isel(latitude=slice(None, None, -1))
ds_ifs_merged = ds_ifs_merged.rename({'number': 'ensemble'})

In [8]:
ds_ifs_train = ds_ifs_merged.sel(time=slice('2017-01-01 00:00', '2018-12-31 12:00'))
ds_ifs_train.to_zarr(
    '../data/processed/ifs/ds_train',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)

In [9]:
ds_ifs_test = ds_ifs_merged.sel(time=slice('2019-01-01 00:00', '2019-12-31 12:00'))
ds_ifs_test.to_zarr(
    '../data/processed/ifs/ds_test',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)